In [6]:
#!pip install torch
!pip install linear_attention_transformer

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/44.6 kB ? eta -:--:--
   ---------------------------------------- 44.6/44.6 kB 2.3 MB/s eta 0:00:00
  Created wheel for axial-positional-embedding: filename=axial_positional_embedding-0.2.1-py3-none-any.whl size=2891 sha256=d64de2b61f572481f7ac6a7e5696686d7766dd997fca9e435b8c3d5cfaf35ab6
  Stored in directory: c:\users\fuad\appdata\local\pip\cache\wheels\40\73\a5\19052fc842d96dcd53f9b447ea0ea0e9d3ea023bbf23359424
Successfully built axial-positional-embedding


In [26]:
import torch

x = torch.rand(5, 3)
print(x)

tensor([[0.7533, 0.5982, 0.1772],
        [0.4211, 0.4495, 0.3219],
        [0.8895, 0.1103, 0.0210],
        [0.5799, 0.5366, 0.5065],
        [0.3254, 0.4447, 0.9873]])


In [6]:
import argparse
import torch
import datetime
import json
import yaml
import os
import pandas as pd
import pickle

from dataset_pm25 import get_dataloader
from main_model import CSDI_PM25
from utils import train, evaluate

In [23]:
parser = argparse.ArgumentParser(description="CSDI")


In [24]:
parser.add_argument("--config", type=str, default="base.yaml")
parser.add_argument('--device', default='cuda:0', help='Device for Attack')
parser.add_argument("--modelfolder", type=str, default="")
parser.add_argument(
    "--targetstrategy", type=str, default="mix", choices=["mix", "random", "historical"]
)
parser.add_argument(
    "--validationindex", type=int, default=0, help="index of month used for validation (value:[0-7])"
)
parser.add_argument("--nsample", type=int, default=100)
parser.add_argument("--unconditional", action="store_true")

_StoreTrueAction(option_strings=['--unconditional'], dest='unconditional', nargs=0, const=True, default=False, type=None, choices=None, required=False, help=None, metavar=None)

In [25]:
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--config CONFIG] [--device DEVICE] [--modelfolder MODELFOLDER]
                             [--targetstrategy {mix,random,historical}] [--validationindex VALIDATIONINDEX]
                             [--unconditional]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\fuad\AppData\Roaming\jupyter\runtime\kernel-95fcd535-846e-426a-b941-79f80f6af715.json


SystemExit: 2

In [18]:




print(args)

usage: ipykernel_launcher.py [-h] [--config CONFIG] [--device DEVICE] [--modelfolder MODELFOLDER]
                             [--targetstrategy {mix,random,historical}] [--validationindex VALIDATIONINDEX]
                             [--nsample NSAMPLE] [--unconditional]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\fuad\AppData\Roaming\jupyter\runtime\kernel-95fcd535-846e-426a-b941-79f80f6af715.json


SystemExit: 2

In [ ]:
path = "config/" + args.config
with open(path, "r") as f:
    config = yaml.safe_load(f)

config["model"]["is_unconditional"] = args.unconditional
config["model"]["target_strategy"] = args.targetstrategy

print(json.dumps(config, indent=4))

In [3]:
import pickle
path = "./data/pm25/pm25_meanstd.pk"
with open(path, 'rb') as f:
    data = pickle.load(f)
data

[array([ 88.04281854,  77.92938209,  88.56736666,  88.71378165,
         98.71179764,  88.67111445,  89.36961409,  93.21280037,
         88.86709098,  96.99126638, 102.50229508,  91.19050311,
         90.09942497,  96.59541324,  88.95387244,  95.17416329,
        103.52942413, 103.54660748, 107.85733216,  87.89406221,
         98.73358746,  93.06996916,  84.6442953 ,  79.11060692,
         75.75927846,  79.2323275 ,  74.97939718,  71.02866175,
         70.17915377,  74.51735016,  63.95535714,  60.81630094,
         75.2570829 , 109.88911381,  99.49978392, 124.17644474]),
 array([ 79.7104599 ,  74.60365778,  77.44130387,  80.93824204,
         90.55020926,  80.73716576,  79.57183029,  83.89937656,
         81.31281883,  87.5163351 ,  88.54831725,  86.09224228,
         83.57440189,  92.36467601,  78.67393445,  85.62109557,
         95.66191451,  92.05832841,  97.94406765,  78.14387384,
         87.42631703,  85.92324398,  77.1344752 ,  77.9505161 ,
         70.83870007,  72.10260033,  6

In [7]:
df = pd.read_csv(
    "./data/pm25/Code/STMVL/SampleData/pm25_ground.txt",
    index_col="datetime",
    parse_dates=True,
)
df.describe()

,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
count,7388.000000,7441.000000,8202.000000,7499.000000,7512.000000,8140.000000,7832.000000,7002.000000,8147.000000,7503.000000,...,7958.000000,7437.000000,7549.000000,8194.000000,7478.000000,7186.000000,7394.000000,7087.000000,6898.000000,7069.000000
mean,80.778154,72.105228,82.024019,83.282304,94.015309,81.450614,83.340398,86.009426,82.965386,90.999067,...,69.929756,66.331720,65.980395,67.609104,59.530088,58.153493,69.753449,103.423451,99.623804,121.347715
std,75.446483,70.488158,72.574603,77.068382,87.615338,74.941528,74.759467,78.284668,76.428544,83.066265,...,66.229909,68.063576,63.577960,71.735021,59.248960,60.466758,62.850682,95.728367,91.799252,110.752999
min,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
25%,24.000000,17.000000,27.000000,26.000000,30.000000,24.000000,26.000000,26.000000,23.000000,28.000000,...,18.000000,15.000000,18.000000,14.000000,16.000000,12.000000,20.000000,36.000000,33.000000,42.000000
50%,57.000000,50.000000,62.000000,60.000000,68.000000,60.000000,64.000000,65.000000,62.000000,69.000000,...,50.000000,45.000000,45.000000,42.000000,37.000000,37.000000,52.000000,76.000000,74.000000,86.000000
75%,116.000000,106.000000,117.000000,120.000000,131.000000,116.000000,116.000000,119.000000,117.000000,124.000000,...,101.000000,94.000000,94.000000,96.000000,85.000000,86.000000,101.000000,138.000000,135.000000,164.000000
max,429.000000,470.000000,500.000000,500.000000,500.000000,482.000000,496.000000,500.000000,500.000000,500.000000,...,500.000000,500.000000,416.000000,442.000000,440.000000,392.000000,500.000000,500.000000,500.000000,500.000000


In [8]:
df


,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
datetime,,,,,,,,,,,,,,,,,,,,,
2014-05-01 01:00:00,138.0,89.0,105.0,98.0,109.0,87.0,88.0,91.0,87.0,87.0,...,101.0,84.0,117.0,NaN,97.0,87.0,74.0,94.0,112.0,109.0
2014-05-01 02:00:00,124.0,85.0,121.0,107.0,101.0,99.0,105.0,102.0,103.0,94.0,...,100.0,77.0,109.0,78.0,97.0,84.0,84.0,101.0,123.0,114.0
2014-05-01 03:00:00,127.0,88.0,130.0,115.0,102.0,109.0,114.0,108.0,112.0,109.0,...,103.0,90.0,105.0,77.0,103.0,83.0,100.0,112.0,143.0,126.0
2014-05-01 04:00:00,129.0,100.0,137.0,123.0,108.0,118.0,118.0,109.0,117.0,111.0,...,110.0,94.0,105.0,90.0,107.0,88.0,103.0,120.0,138.0,130.0
2014-05-01 05:00:00,119.0,109.0,144.0,129.0,115.0,124.0,130.0,116.0,124.0,114.0,...,105.0,80.0,104.0,83.0,111.0,85.0,108.0,125.0,145.0,137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-04-30 19:00:00,86.0,72.0,70.0,72.0,73.0,63.0,65.0,69.0,74.0,66.0,...,130.0,136.0,79.0,69.0,74.0,125.0,94.0,67.0,64.0,79.0
2015-04-30 20:00:00,87.0,73.0,73.0,76.0,83.0,63.0,61.0,62.0,62.0,68.0,...,133.0,124.0,64.0,67.0,75.0,124.0,128.0,103.0,69.0,77.0
2015-04-30 21:00:00,80.0,74.0,80.0,70.0,84.0,69.0,72.0,75.0,77.0,73.0,...,117.0,123.0,72.0,70.0,82.0,125.0,139.0,166.0,78.0,94.0


In [9]:
test_month = [3, 6, 9, 12]
for i in test_month:
    df = df[df.index.month != i]
    
df

,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
datetime,,,,,,,,,,,,,,,,,,,,,
2014-05-01 01:00:00,138.0,89.0,105.0,98.0,109.0,87.0,88.0,91.0,87.0,87.0,...,101.0,84.0,117.0,NaN,97.0,87.0,74.0,94.0,112.0,109.0
2014-05-01 02:00:00,124.0,85.0,121.0,107.0,101.0,99.0,105.0,102.0,103.0,94.0,...,100.0,77.0,109.0,78.0,97.0,84.0,84.0,101.0,123.0,114.0
2014-05-01 03:00:00,127.0,88.0,130.0,115.0,102.0,109.0,114.0,108.0,112.0,109.0,...,103.0,90.0,105.0,77.0,103.0,83.0,100.0,112.0,143.0,126.0
2014-05-01 04:00:00,129.0,100.0,137.0,123.0,108.0,118.0,118.0,109.0,117.0,111.0,...,110.0,94.0,105.0,90.0,107.0,88.0,103.0,120.0,138.0,130.0
2014-05-01 05:00:00,119.0,109.0,144.0,129.0,115.0,124.0,130.0,116.0,124.0,114.0,...,105.0,80.0,104.0,83.0,111.0,85.0,108.0,125.0,145.0,137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-04-30 19:00:00,86.0,72.0,70.0,72.0,73.0,63.0,65.0,69.0,74.0,66.0,...,130.0,136.0,79.0,69.0,74.0,125.0,94.0,67.0,64.0,79.0
2015-04-30 20:00:00,87.0,73.0,73.0,76.0,83.0,63.0,61.0,62.0,62.0,68.0,...,133.0,124.0,64.0,67.0,75.0,124.0,128.0,103.0,69.0,77.0
2015-04-30 21:00:00,80.0,74.0,80.0,70.0,84.0,69.0,72.0,75.0,77.0,73.0,...,117.0,123.0,72.0,70.0,82.0,125.0,139.0,166.0,78.0,94.0


In [ ]:
    def create_normalizer_pm25():
        df = pd.read_csv(
            "./data/pm25/Code/STMVL/SampleData/pm25_ground.txt",
            index_col="datetime",
            parse_dates=True,
        )
        test_month = [3, 6, 9, 12]
        for i in test_month:
            df = df[df.index.month != i]
        mean = df.describe().loc["mean"].values
        std = df.describe().loc["std"].values
        path = "./data/pm25/pm25_meanstd.pk"
        with open(path, "wb") as f:
            pickle.dump([mean, std], f)
    create_normalizer_pm25()